# TASK 1: Write a DL model using a pytorch framework and export to onnx , do onnx simplification (resnet18)

In [ ]:
!pip install onnx
!pip install onnxsim

In [ ]:
!pip install torch
!pip install torchvision

In [ ]:
import torch
import torch.onnx
import onnx
import onnxsim
from torchvision import models

In [ ]:
# Load Pretrained ResNet-18 Model

model = models.resnet18(pretrained=True)
model.eval()  # Set to evaluation mode

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 125MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [ ]:
# Input for Export (Batch Size, Color Channels, Image pixels)
dummy_input = torch.randn(1, 3, 224, 224)

In [ ]:
# Define ONNX Export Parameters
onnx_filename = "resnet18.onnx"
torch.onnx.export(
    model,              # Model to export
    dummy_input,        # Input tensor
    onnx_filename,      # Output filename
    input_names=["input"],  # Input tensor name
    output_names=["output"], # Output tensor name
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}, # Allow batch size flexibility
    opset_version=11    # ONNX Opset version
)
print(f"Model exported to {onnx_filename}")

Model exported to resnet18.onnx


In [ ]:
# Load and Verify ONNX Model
onnx_model = onnx.load(onnx_filename)  # Load model
onnx.checker.check_model(onnx_model)   # Validate model
print("ONNX model is valid!")

ONNX model is valid!


In [ ]:
# Checking if the file is simplifiable
model_path = "resnet18.onnx"
simplified_model_path = "resnet18_simplified.onnx"

# Load ONNX model
model = onnx.load(model_path)

# Simplify model with debugging enabled
simplified_model, check = onnxsim.simplify(model, check_n=3)

if not check:
    print("\nSimplification check failed! The model may not be simplified.")
else:
    print("\nSimplification successful!")

# Save the new model
onnx.save(simplified_model, simplified_model_path)
print(f"\nSimplified model saved as {simplified_model_path}")

Checking 0/3...
shape[0] of input "input" is dynamic, we assume it presents batch size and set it as 1 when testing. If it is not wanted, please set the it manually by --test-input-shape (see `onnxsim -h` for the details).
Checking 1/3...
shape[0] of input "input" is dynamic, we assume it presents batch size and set it as 1 when testing. If it is not wanted, please set the it manually by --test-input-shape (see `onnxsim -h` for the details).
Checking 2/3...
shape[0] of input "input" is dynamic, we assume it presents batch size and set it as 1 when testing. If it is not wanted, please set the it manually by --test-input-shape (see `onnxsim -h` for the details).
Simplification successful!
Simplified model saved as resnet18_simplified.onnx


#COMPARING SIMPLIFIED MODEL AND UNSIMPLIFIED PRETRAINED MODEL

In [ ]:
import os
import time
import numpy as np
import onnxruntime as ort

# Load original and simplified ONNX models
original_model = "resnet18.onnx"
simplified_model = "resnet18_simplified.onnx"

# Compare file sizes
original_size = os.path.getsize(original_model) / 1024  # KB
simplified_size = os.path.getsize(simplified_model) / 1024  # KB
print(f"Original Model Size: {original_size:.2f} KB")
print(f"Simplified Model Size: {simplified_size:.2f} KB")

# Load ONNX models into ONNX Runtime
sess_orig = ort.InferenceSession(original_model)
sess_simp = ort.InferenceSession(simplified_model)

# Create a dummy input (batch size 1)
input_name = sess_orig.get_inputs()[0].name
dummy_input = np.random.randn(1, 3, 224, 224).astype(np.float32)

# Measure inference time
start = time.time()
sess_orig.run(None, {input_name: dummy_input})
orig_time = time.time() - start

start = time.time()
sess_simp.run(None, {input_name: dummy_input})
simp_time = time.time() - start

print(f"Original Model Inference Time: {orig_time:.5f} sec")
print(f"Simplified Model Inference Time: {simp_time:.5f} sec")

# Compare number of nodes
orig_nodes = len(onnx.load(original_model).graph.node)
simp_nodes = len(onnx.load(simplified_model).graph.node)
print(f"Original Model Nodes: {orig_nodes}")
print(f"Simplified Model Nodes: {simp_nodes}")

Original Model Size: 45652.90 KB
Simplified Model Size: 45656.33 KB
Original Model Inference Time: 0.05338 sec
Simplified Model Inference Time: 0.05951 sec
Original Model Nodes: 49
Simplified Model Nodes: 49


Output => Not much of a difference between the original and the simplifies model
#Inference:
1. Model Already Optimized: If the ONNX model is already exported in an efficient format (e.g., via PyTorch’s ONNX exporter with optimizations enabled), onnxsim may not find redundant nodes to remove.

2. Redundant Nodes Not Present: The simplifier mainly removes unnecessary Transpose, Reshape, and Identity layers. If your model doesn’t have these, there won’t be much change.

3. Constant Folding Already Done: If all constant expressions were already precomputed in the original model, simplification won’t further optimize it.

In [ ]:
# Load Un-Pretrained ResNet-18 Model
model = models.resnet18(pretrained=False)
model.eval()  # Set to evaluation mode

# Input for Export (Batch Size, Color Channels, Image Size)
dummy_input = torch.randn(1, 3, 224, 224)

# Define ONNX Export Parameters
onnx_filename = "resnet18.onnx"
torch.onnx.export(
    model,              # Model to export
    dummy_input,        # Input tensor
    onnx_filename,      # Output filename
    input_names=["input"],  # Input tensor name
    output_names=["output"], # Output tensor name
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}, # Allow batch size flexibility
    opset_version=11    # ONNX Opset version
)
print(f"Model exported to {onnx_filename}")

# Load and Verify ONNX Model
onnx_model = onnx.load(onnx_filename)  # Load model
onnx.checker.check_model(onnx_model)   # Validate model
print("ONNX model is valid!")

# Simplify the ONNX Model
simplified_model, check = onnxsim.simplify(onnx_model)
assert check, "Simplified ONNX model is incorrect!"

simplified_filename = "resnet18_simplified.onnx"
onnx.save(simplified_model, simplified_filename)
print(f"Simplified model saved as {simplified_filename}")


Model exported to resnet18.onnx
ONNX model is valid!
Simplified model saved as resnet18_simplified.onnx


#COMPARING SIMPLIFIED MODEL AND UNSIMPLIFIED UN-PRETRAINED MODEL

In [ ]:
# Load original and simplified ONNX models
original_model = "resnet18.onnx"
simplified_model = "resnet18_simplified.onnx"

# Compare file sizes
original_size = os.path.getsize(original_model) / 1024  # KB
simplified_size = os.path.getsize(simplified_model) / 1024  # KB
print(f"Original Model Size: {original_size:.2f} KB")
print(f"Simplified Model Size: {simplified_size:.2f} KB")

# Load ONNX models into ONNX Runtime
sess_orig = ort.InferenceSession(original_model)
sess_simp = ort.InferenceSession(simplified_model)

# Create a dummy input (batch size 1)
input_name = sess_orig.get_inputs()[0].name
dummy_input = np.random.randn(1, 3, 224, 224).astype(np.float32)

# Measure inference time
start = time.time()
sess_orig.run(None, {input_name: dummy_input})
orig_time = time.time() - start

start = time.time()
sess_simp.run(None, {input_name: dummy_input})
simp_time = time.time() - start

print(f"Original Model Inference Time: {orig_time:.5f} sec")
print(f"Simplified Model Inference Time: {simp_time:.5f} sec")

# Compare number of nodes
orig_nodes = len(onnx.load(original_model).graph.node)
simp_nodes = len(onnx.load(simplified_model).graph.node)
print(f"Original Model Nodes: {orig_nodes}")
print(f"Simplified Model Nodes: {simp_nodes}")

Original Model Size: 45638.36 KB
Simplified Model Size: 45640.92 KB
Original Model Inference Time: 0.05495 sec
Simplified Model Inference Time: 0.05611 sec
Original Model Nodes: 65
Simplified Model Nodes: 49


#Inference:
The un-pretrained model has variation in the number of model nodes!